In [10]:
# Install
# conda install -c conda-forge selenium
# conda install -c clinicalgraphics/label/archived pyvirtualdisplay 
# conda install -c anaconda boto
# conda install -c conda-forge httplib2

#!/usr/bin/python
#-*- coding: utf-8 -*-

# Install
# apt-get install xvfb python-imaging (firefox or chromium-chromedriver)
# pip install selenium pyvirtualdisplay boto

import sys
import os
import socket
import logging
import datetime
import time
import httplib2

from io import StringIO
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

from pyvirtualdisplay.xvfb import XvfbDisplay
from PIL import Image
from boto import ses


from matplotlib import pyplot as plt

#reload(sys)
#sys.setdefaultencoding("utf-8")
socket.setdefaulttimeout(60)

logging.basicConfig(level=logging.INFO)
log = logging.getLogger('shaco')


# *****************************************************

class Shaco():
    
    browser = None
    display = None

    def __init__(self):
        self.display = XvfbDisplay(size=(1024, 768), fbdir="c:\tmp").start()
        
        

        self.browser = webdriver.Chrome()

        # Chrome
        #CHROMIUM_PATH = '/usr/lib/chromium-browser'
        #os.environ['LD_LIBRARY_PATH'] = ':'.join(['/'.join([CHROMIUM_PATH, '/libs']), os.environ.get('LD_LIBRARY_PATH', '.')])
        #self.browser = webdriver.Chrome(executable_path='/'.join([CHROMIUM_PATH, 'chromedriver']))


    def __del__(self):
        if self.browser: self.browser.quit()
        if self.display: self.display.stop()
            
    def browse_action(self):
        
        self.browser.get("http://www.tjrs.jus.br/site/")


        elem = self.browser.find_element_by_id("busca") # encontra o campo de busca
        elem.clear() # limpa o input
        elem.send_keys("processo") # escreve "processo" no campo
        elem.send_keys(Keys.RETURN) # aperta enter

        elem = self.browser.find_element_by_link_text("Consulta Processual") # seleciona a aba "Consulta Processual"
        elem.send_keys(Keys.RETURN)


        # como o formulário é carregado a partir de um iframe,
        # troca para o iframe onde estão os campos de pesquisa
        iframe = self.browser.find_element_by_xpath("//iframe[@id='iframeBusca']")
        self.browser.switch_to.frame(iframe)

        # executa o javascript para abrir a aba de consulta por nome da parte
        self.browser.execute_script("AlternarAbas('td_parte','div_parte')")

        # seleciona "Estrela" no select TJ/Comarca
        self.browser.find_element_by_xpath("//select[@name='id_comarca3']/option[text()='Estrela']").click()

        # preenche o nome da parte
        nome_da_parte = self.browser.find_element_by_xpath("//input[@name='nome_parte']")
        nome_da_parte.send_keys("Francisco Schwertner")

        # clica no botão pesquisar
        btn_pesquisar = self.browser.find_element_by_xpath("//input[@name='btnPesquisar']")
        btn_pesquisar.send_keys(Keys.RETURN)


        self.browser.switch_to.default_content()
        
        #self.browser.close()
        
            
    def get_screenshot_with_chrome(self):
        totalWidth = self.browser.execute_script('return document.documentElement.scrollWidth');
        totalHeight = self.browser.execute_script('return document.documentElement.scrollHeight');

        viewportWidth = self.browser.execute_script('return document.documentElement.clientWidth');
        viewportHeight = self.browser.execute_script('return document.documentElement.clientHeight');

        rectangles = []
        for y in range(0, totalHeight, viewportHeight):
            newHeight = viewportHeight
            if (y + viewportHeight) > totalHeight:
                newHeight = totalHeight - y

            for x in range(0, totalWidth, viewportWidth):
                newWidth = viewportWidth
                if (x + viewportWidth) > totalWidth:
                    newWidth = totalWidth - x

                rectangle = Rectangle(x, y, x+newWidth, y+newHeight)
                rectangles.append(rectangle)


        im = Image.new('RGB', (totalWidth, totalHeight))
        prevRectangle = None
        for rectangle in rectangles:
            if prevRectangle:
                xDiff = rectangle.right - prevRectangle.right
                yDiff = rectangle.lower - prevRectangle.lower
                self.browser.execute_script('window.scrollBy({}, {})'.format(xDiff, yDiff))

            part = Image.open(StringIO(self.browser.get_screenshot_as_png()))
            part = part.crop((0, 0, viewportWidth, viewportHeight))
            box = (rectangle.right - part.size[0], rectangle.lower - part.size[1], rectangle.right, rectangle.lower)
            im.paste(part, box)

            prevRectangle = rectangle

        stream = StringIO()
        im.save(stream, 'png')
        screenshot = stream.getvalue()
        stream.close()
        return screenshot
# *****************************************************


if __name__ == '__main__':
    shaco = None
    name = None

    try:
        shaco = Shaco()
        shaco.browse_action()
        imagem = shaco.get_screenshot_with_chrome()
        
        plt.imshow(imagem)
        plt.xticks([])
        plt.yticks([])
        plt.title('my picture')
        plt.show()
        
        
    except Exception:
        #exception_handler()
        raise
    finally:
        del shaco




FileNotFoundError: [WinError 3] O sistema não pode encontrar o caminho especificado: '/tmp'